In [1]:
import os

import torch
import torchvision

In [2]:
NUM_WORKERS = os.cpu_count()
NUM_WORKERS

12

In [3]:
import os
import requests
import zipfile
from pathlib import Path

# Setup path to data folder
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

# If the image folder doesn't exist, download it and prepare it... 
if image_path.is_dir():
    print(f"{image_path} directory exists.")
else:
    print(f"Did not find {image_path} directory, creating one...")
    image_path.mkdir(parents=True, exist_ok=True)

# Download pizza, steak, sushi data
with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
    request = requests.get(
        "https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
    print("Downloading pizza, steak, sushi data...")
    f.write(request.content)

# Unzip pizza, steak, sushi data
with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
    print("Unzipping pizza, steak, sushi data...") 
    zip_ref.extractall(image_path)

# Remove zip file
os.remove(data_path / "pizza_steak_sushi.zip")

Did not find data/pizza_steak_sushi directory, creating one...
Unzipping pizza, steak, sushi data...


In [4]:
# The resulting data folder looks like the following

## Create Datasets and DataLoaders
data_setup.py

In [5]:
"""
Contains functionality for creating PyTorch DataLoaders for 
images classification data.
"""
import os

import torch
import torchvision

NUM_WORKERS = os.cpu_count()


def create_dataloaders(
    train_dir: str,
    test_dir: str,
    transform: torchvision.transforms.Compose,
    batch_size: int = 32,
    num_workers: int = NUM_WORKERS,
):
    """
    Creates training and testing DataLoaders.

    Takes in training/testing directory paths and turns them into
    PyTorch Datasets and then DataLoaders.

    Args:

    Returns:
        A tuple of (train_dataloader, test_dataloader, class_names) where
        class_names is a list of target classes.

    Example:
        train_dataloader, test_dataloader, class_names =
         create_dataloaders(
            tr_dir, tst_dir, transform, batch_size, num_workers)
    """
    # Use ImageFolder to create dataset(s)
    train_data = torchvision.datasets.ImageFolder(train_dir, transform=transform)
    test_data = torchvision.datasets.ImageFolder(test_dir, transform=transform)
    
    class_names = train_data.classes
    
    # Turn images to dataloaders
    train_dataloader = torch.utils.data.DataLoader(
        train_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True
    )
    
    test_dataloader = torch.utils.data.DataLoader(
        test_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True
    )
    
    return train_dataloader, test_dataloader, class_names

In [6]:
# Test data_setup

In [7]:
data_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((64, 64)),
    torchvision.transforms.ToTensor()
])


In [8]:
train_dataloader, test_dataloader, class_names = create_dataloaders(
    "data/pizza_steak_sushi/train",
    "data/pizza_steak_sushi/test",
    data_transform
)

In [9]:
class_names

['pizza', 'steak', 'sushi']

In [10]:
X,y = next(iter(train_dataloader))

In [11]:
type(X), X.shape, y

(torch.Tensor,
 torch.Size([32, 3, 64, 64]),
 tensor([1, 1, 2, 1, 1, 0, 1, 0, 1, 1, 2, 2, 0, 0, 2, 1, 0, 2, 1, 2, 0, 2, 0, 0,
         2, 2, 2, 0, 1, 1, 2, 1]))

## Build model
model_builder.py

In [12]:
"""
Contains PyTorch model code to instantiate a TinyVGG model.
"""
import torch
from torch import nn

class TinyVGG(nn.Module):
    """
    Creates the TinyVGG architecture.
    
    Args:
        input_shape: an int indicating num of input channels
        hidden_units: an int indicating num of hidden units between layers
        output_shape: an int indicating num of output units
    """
    def __init__(self, input_shape:int, hidden_units:int, output_shape:int):
        super().__init__()
        
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(
                in_channels=input_shape,
                out_channels=hidden_units,
                kernel_size=3,
                stride=1,
                padding=0
            ),
            nn.ReLU(),
            nn.Conv2d(
                in_channels=hidden_units,
                out_channels=hidden_units,
                kernel_size=3,
                stride=1,
                padding=0
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
            nn.ReLU(),
            nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units*13*13, out_features=output_shape)
        )
            
    def forward(self, x:torch.Tensor):
        #x = self.conv_block_1(x)
        #x = self.conv_block_2(x)
        #x = self.classifier(x)
        return self.classifier(self.conv_block_2(self.conv_block_1(x)))

In [13]:
# Test above code

In [14]:
import utils
device = utils.get_device()
device

'mps'

In [15]:
X.shape, y, len(class_names)

(torch.Size([32, 3, 64, 64]),
 tensor([1, 1, 2, 1, 1, 0, 1, 0, 1, 1, 2, 2, 0, 0, 2, 1, 0, 2, 1, 2, 0, 2, 0, 0,
         2, 2, 2, 0, 1, 1, 2, 1]),
 3)

In [16]:
torch.manual_seed(123)

model = TinyVGG(input_shape=3, hidden_units=10, output_shape=3).to(device)
model

TinyVGG(
  (conv_block_1): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1690, out_features=3, bias=True)
  )
)

## Create train/test functions
- train_step(): training ops per epoch
- test_step(): testing ops per epoch
- train(): for epoch in epochs, calling train_step() and test_step()

engine.py

In [18]:
"""
Contains functions for training/testing a PyTorch model.
"""
import torch

from tqdm.auto import tqdm
from typing import Dict, List, Tuple

In [20]:
def train_step(
    model: torch.nn.Module,
    dataloader: torch.utils.data.DataLoader,
    loss_fn: torch.nn.Module,
    optimizer: torch.optim.Optimizer,
    device: torch.device
) -> Tuple[float, float]:
    """
    Trains a PyTorch model per epoch.
    
    Sets a target PyTorch model to "train" mode and then steps through the 
    forward/backward pass: forward, loss, loss backward, optim i.e. gradient descent.
    
    Args:
        ***
    
    Returns:
        A tuple of (training loss, training accuracy), e.g., (0.1111, 0.8765)
    """
    model.train()
    
    train_loss, train_acc = 0.0, 0.0
    
    # Loop through the batches in given DataLoader
    for batch, (X,y) in enumerate(dataloader):
        X,y = X.to(device), y.to(device)
        
        # Forward
        y_pred = model(X)
        
        # Compute loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()
        
        # Loss backward
        optimizer.zero_grad()
        loss.backward()
        
        # Grad descent
        optimizer.step()
        
        # Metrics
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item() / len(y)
    
    # Modify accumulated (over all batches) metrics to be avg metrics per batch
    train_loss /= len(dataloader)
    train_acc /= len(dataloader)
    
    return train_loss, train_acc

In [21]:
def test_step(
    model: torch.nn.Module,
    dataloader: torch.utils.data.DataLoader,
    loss_fn: torch.nn.Module,
    device: torch.device
) -> Tuple[float, float]:
    """
    Tests a PyTorch model per epoch.
    
    Sets a target PyTorch model to "eval" mode and then perform the 
    forward step.
    
    Args:
        ***
    
    Returns:
        A tuple of (test loss, test accuracy), e.g., (0.1111, 0.8765)
    """
    model.eval()
    
    test_loss, test_acc = 0.0, 0.0
    
    with torch.inference_mode():
        # Loop through the batches in given DataLoader
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)
            
            # Forward
            test_pred_logits = model(X)
            
            # Compute loss
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()
            
            # Compute accuracy
            test_pred_labels = test_pred_logits.argmax(dim=1)
            test_acc += (test_pred_labels == y).sum().item() / len(test_pred_labels)
        
        # Modify accumulated metrics to be avg metrics per batch
        test_loss /= len(dataloader)
        test_acc /= len(dataloader)
        
        return test_loss, test_acc

In [31]:
def train(
    model: torch.nn.Module,
    train_dataloader: torch.utils.data.DataLoader,
    test_dataloader: torch.utils.data.DataLoader,
    optimizer: torch.optim.Optimizer,
    loss_fn: torch.nn.Module,
    epochs: int,
    device: torch.device
) -> Dict[str, List]:
    """
    Defines the training loop over multiple epochs.
    
    Args:
        ***
    
    Returns:
        A dict of {
        train_loss: [...], # List constains values over all epochs
        train_acc: [...], 
        test_loss: [...], 
        test_acc: [...]}
    """
    results = {
        "train_loss":[], "train_acc":[], 
        "test_loss":[], "test_acc":[]}
    
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(
            model=model,
            dataloader=train_dataloader,
            loss_fn=loss_fn,
            optimizer=optimizer,
            device=device
        )
        
        test_loss, test_acc = test_step(
            model=model,
            dataloader=test_dataloader,
            loss_fn=loss_fn,
            device=device
        )
        
        print(
            f"Epoch {epoch + 1}:",
            f"Train loss: {train_loss:.4f}, Train acc: {train_acc:.4f} | ",
            f"Test loss: {test_loss:.4f}, Test acc: {test_acc:.4f}"
        )
    
    return results

In [29]:
# Test above functions

In [30]:
from timeit import default_timer as timer 
start_time = timer()
    
torch.manual_seed(123)

NUM_EPOCHS = 3

model0 = TinyVGG(input_shape=3, hidden_units=10, output_shape=len(class_names)).to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model0.parameters(), lr=0.001)

model0_results = train(
    model=model0,
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    optimizer=optimizer,
    loss_fn=loss_fn,
    epochs=NUM_EPOCHS,
    device=device
)

end_time = timer()
print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch 1: Train loss: 1.1014, Train acc: 0.2930 |  Test loss: 1.1088, Test acc: 0.3201
Epoch 2: Train loss: 1.0998, Train acc: 0.3008 |  Test loss: 1.1113, Test acc: 0.2775
Epoch 3: Train loss: 1.0869, Train acc: 0.4648 |  Test loss: 1.1106, Test acc: 0.3523
[INFO] Total training time: 172.751 seconds


## Save model
utils.py

In [32]:
import pathlib

def save_model(
    model: torch.nn.Module,
    target_dir: str,
    model_name: str
):
    """
    Saves a PyTorch model to disk at target directory.
    
    Args:
        ***
        
    Example:
        save_model(model0, "models/", "05_modular_tiny_vgg.pt")
    """
    target_dir_path = pathlib.Path(target_dir)
    target_dir_path.mkdir(parents=True, exist_ok=True)
    
    model_save_path = target_dir_path/model_name
    
    print(f"[INFO] Saving model to {model_save_path}")
    torch.save(obj=model.state_dict(), f=model_save_path)

In [33]:
# Test above function

In [34]:
save_model(model=model0, target_dir="models", model_name="05_modular_tiny_vgg.pt")

[INFO] Saving model to models/05_modular_tiny_vgg.pt


## Now we have the functional elements. Need to structure the code base